In [1]:
#import dependencies 
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np

import math  

import seaborn as sns
sns.set(style="darkgrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

In [251]:
#check list of files in data 
print(os.listdir("./data/open_cloud"))

['m5d_open_cloud_20x_y-cruncher_11-14-2019.csv', 'm5d_open_cloud_30x_sysbench_11-14-2019.csv', '.DS_Store', 'm5d_open_cloud_30x_y-cruncher_11-14-2019.csv', '50x_m5d_iperf_11-16-2019.csv', '50x_m5d_y-cruncher_open_cloud_11-16-2019.csv', '50x_m5d_sysbench_open_cloud_11-16-2019-us-east-1b-2.csv', '50x_m5d_pgbench.csv', 'm5d_open_cloud_20x_sysbench_11-14-2019.csv', 'm5d_open_cloud_30x_pgbench_11-14-2019.csv']


In [248]:
data_name = '50x_m5d_iperf_11-16-2019.csv'

In [249]:
df_data = pd.read_csv('./data/open_cloud/' + data_name);

In [250]:
df_data.count()

1573960552-0                                                                23519
i-0d6feb91baa545931                                                         23519
m5d.large                                                                   23519
9.157414198                                                                 23519
--test=cpu --cpu-max-prime=2000000 --num-threads=2 --max-requests=10 run    23519
1802.76ms                                                                   23519
9.0154s                                                                     23519
2                                                                           23519
dtype: int64

In [225]:
#iperf remove trailing text
df_data['bandwidthUpload'] = df_data['bandwidthUpload'].str.split(" ").str[0].astype('float64')
df_data['bandwitdhDownload'] = df_data['bandwitdhDownload'].str.split(" ").str[0].astype('float64')
df_data.dropna(inplace = True)

In [226]:
df_data.head()

,instanceID,instanceType,experimentID,interval,bandwidthUpload,bandwitdhDownload,setId,vmId,cmd
0,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,2.56,14,14,iperf -c 172.31.32.213 --dualtest --window 416...
1,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,4.97,14,14,iperf -c 172.31.32.213 --dualtest --window 416...
2,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,4.79,14,14,iperf -c 172.31.32.213 --dualtest --window 416...
3,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,2.54,14,14,iperf -c 172.31.32.213 --dualtest --window 416...
4,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,4.94,14,14,iperf -c 172.31.32.213 --dualtest --window 416...


In [227]:
df_data.describe()

,experimentID,bandwidthUpload,bandwitdhDownload,setId,vmId
count,1.357900e+04,13579.000000,13579.000000,13579.000000,13579.000000
mean,1.573945e+09,573.892784,538.667081,33.461595,17.170631
std,0.000000e+00,315.354298,334.598535,11.978102,11.913129
min,1.573945e+09,1.000000,1.000000,1.000000,1.000000
25%,1.573945e+09,742.000000,7.505000,25.000000,7.000000
50%,1.573945e+09,744.000000,726.000000,35.000000,15.000000
75%,1.573945e+09,745.000000,746.000000,44.000000,26.000000
max,1.573945e+09,979.000000,1000.000000,50.000000,50.000000


In [230]:
#adjust megabite values down assuming all mbs values are over 500mb
df_data['bandwitdhDownload'] = np.where(df_data['bandwitdhDownload'] < 500, df_data['bandwitdhDownload'], df_data['bandwitdhDownload']/1000)
df_data['bandwidthUpload'] = np.where(df_data['bandwidthUpload'] < 500, df_data['bandwidthUpload'], df_data['bandwidthUpload']/1000)

In [232]:
#add total bandwidth column
df_data['total'] = df_data['bandwidthUpload'] + df_data['bandwitdhDownload']

In [233]:
df_data.head()

,instanceID,instanceType,experimentID,interval,bandwidthUpload,bandwitdhDownload,setId,vmId,cmd,total
0,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,2.56,14,14,iperf -c 172.31.32.213 --dualtest --window 416...,7.53
1,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,4.97,14,14,iperf -c 172.31.32.213 --dualtest --window 416...,9.94
2,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,4.79,14,14,iperf -c 172.31.32.213 --dualtest --window 416...,9.76
3,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,2.54,14,14,iperf -c 172.31.32.213 --dualtest --window 416...,7.51
4,i-031b2f76071a565fc,m5d.large,1573945123,15.0 sec,4.97,4.94,14,14,iperf -c 172.31.32.213 --dualtest --window 416...,9.91


In [234]:
#group by instanceID and take the mean of each column
df = df_data.groupby('instanceID').mean()

In [235]:
#groupby instanceID and take the standard deviation each column
df_std = df_data.groupby('instanceID').std()

In [237]:
#add stds to main columns
df['stdUp'] = df_std['bandwidthUpload']
df['stdDown'] = df_std['bandwitdhDownload']
df['stdTotal'] = df_std['total']

In [240]:
df.tail()

,bandwidthUpload,bandwitdhDownload,vmId,total,stdUp,stdDown,stdTotal
instanceID,,,,,,,
i-0ece06eab633a3526,9.433636,6.999091,50.0,16.432727,0.369006,1.087864,1.273602
i-0f0e24c09425eac71,1.252613,1.249976,5.0,2.502589,1.911506,1.907965,3.814109
i-0f83d46715e5b7ba9,2.618579,2.618785,40.0,5.237364,2.062784,2.027959,4.054448
i-0fe52617ed49f921d,1.507298,1.485100,22.0,2.992398,1.561125,1.511888,3.052346
i-0ff32a7bce1e3a734,2.166097,2.156036,36.0,4.322133,3.168969,3.108296,6.255787


In [239]:
#if sysbench
#df.drop(columns =['thread-num'], inplace = True)

#if iperf
df.drop(columns =['setId', 'experimentID'], inplace = True)

In [241]:
#print to csv
df.to_csv(path_or_buf='./data/summary/' + data_name + '-Summary.csv')

In [242]:
df.describe()

,bandwidthUpload,bandwitdhDownload,vmId,total,stdUp,stdDown,stdTotal
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,2.225280,2.170171,25.306122,4.395451,1.763579,2.113180,3.767290
std,1.507233,1.241722,14.663168,2.735766,0.777607,2.810551,2.841313
min,1.215682,1.215093,1.000000,2.430775,0.036442,0.361583,0.372100
25%,1.339538,1.364126,13.000000,2.729621,1.352426,1.319921,2.640873
50%,1.607484,1.653375,25.000000,3.305473,1.700614,1.714594,3.433598
75%,2.413438,2.421956,38.000000,4.835394,2.062784,2.027959,4.054448
max,9.433636,6.999091,50.000000,16.432727,3.939934,20.913419,20.996954


In [52]:
#graph the probablity of getting "better vm"